# Find the model

## Global configurations

In [3]:
import logging
import logging.config
import os
import sys
import warnings
from enum import auto
import pandas as pd
import numpy as np
from IPython.core.display import display

rootdir = '/home/hym/trunk/TCRBert'
workdir = '%s/notebook' % rootdir
datadir = '%s/data' % rootdir
srcdir = '%s/tcrbert' % rootdir
outdir = '%s/output' % rootdir

os.chdir(workdir)

sys.path.append(rootdir)
sys.path.append(srcdir)

from tcrbert.exp import Experiment
from tcrbert.predlistener import PredResultRecoder


# Display
pd.set_option('display.max.rows', 2000)
pd.set_option('display.max.columns', 2000)

# Logger
warnings.filterwarnings('ignore')
logging.config.fileConfig('../config/logging.conf')
logger = logging.getLogger('tcrbert')
logger.setLevel(logging.INFO)

# Target experiment
exp_key = 'testexp'
Experiment.load_exp_conf('testexp', reload=True)
experiment = Experiment.from_key(exp_key)

exp_conf = experiment.exp_conf

display(exp_conf)

2023-11-22 17:26:40 [INFO]: Loaded exp_conf: {'title': 'testexp', 'description': 'testexp', 'paper': 'ASAP', 'model_config': '../config/bert-base/', 'outdir': '../output/testexp', 'train': {'pretrained_model': {'type': 'tape', 'location': '../config/bert-base/'}, 'data_parallel': True, 'backup': 'train.bak.{date}.tar.gz', 'rounds': [{'data': 'dash_vdjdb_mcpas', 'test_size': 0.2, 'batch_size': 256, 'n_epochs': 2, 'n_workers': 12, 'metrics': ['accuracy'], 'optimizer': {'type': 'adamw', 'lr': 0.0001}, 'train_bert_encoders': [-10, None], 'early_stopper': {'monitor': 'accuracy', 'patience': 1}, 'model_checkpoint': {'chk': 'train.{round}.model_{epoch}.chk', 'monitor': 'accuracy', 'save_best_only': True, 'period': 1}, 'result': 'train.{round}.result.json'}, {'data': 'dash_vdjdb_mcpas', 'test_size': 0.2, 'batch_size': 256, 'n_epochs': 2, 'n_workers': 12, 'metrics': ['accuracy'], 'optimizer': {'type': 'adamw', 'lr': 0.0001}, 'train_bert_encoders': [-6, None], 'early_stopper': {'monitor': 'accur

{'title': 'testexp',
 'description': 'testexp',
 'paper': 'ASAP',
 'model_config': '../config/bert-base/',
 'outdir': '../output/testexp',
 'train': {'pretrained_model': {'type': 'tape',
   'location': '../config/bert-base/'},
  'data_parallel': True,
  'backup': 'train.bak.{date}.tar.gz',
  'rounds': [{'data': 'dash_vdjdb_mcpas',
    'test_size': 0.2,
    'batch_size': 256,
    'n_epochs': 2,
    'n_workers': 12,
    'metrics': ['accuracy'],
    'optimizer': {'type': 'adamw', 'lr': 0.0001},
    'train_bert_encoders': [-10, None],
    'early_stopper': {'monitor': 'accuracy', 'patience': 1},
    'model_checkpoint': {'chk': 'train.{round}.model_{epoch}.chk',
     'monitor': 'accuracy',
     'save_best_only': True,
     'period': 1},
    'result': 'train.{round}.result.json'},
   {'data': 'dash_vdjdb_mcpas',
    'test_size': 0.2,
    'batch_size': 256,
    'n_epochs': 2,
    'n_workers': 12,
    'metrics': ['accuracy'],
    'optimizer': {'type': 'adamw', 'lr': 0.0001},
    'train_bert_enc

## Test training

In [7]:
from tcrbert.dataset import TCREpitopeSentenceDataset, CN
from collections import OrderedDict, Counter
from torch.utils.data import DataLoader

eval_ds = TCREpitopeSentenceDataset.from_key('shomuradova')

metrics = ['accuracy', 'f1', 'roc_auc']

# Train
experiment.train()

# Backup the train results
experiment.backup_train_results()

# Get best model and evaluate the model
model = experiment.load_eval_model()
eval_recoder = PredResultRecoder(output_attentions=True, output_hidden_states=True)
model.add_pred_listener(eval_recoder)    
data_loader = DataLoader(eval_ds, batch_size=len(eval_ds), shuffle=False, num_workers=2)
logger.info('Predicting for %s' % eval_ds.name)
model.predict(data_loader=data_loader, metrics=metrics, use_cuda=True)

2023-11-22 17:35:14 [INFO]: shomuradova dataset was loaded from ../output/shomuradova.data.csv, df.shape: (610, 9)
2023-11-22 17:35:14 [INFO]: ======================
2023-11-22 17:35:14 [INFO]: Begin train at 2023-11-22 17:35:14.935758
2023-11-22 17:35:14 [INFO]: Loading the TAPE pretrained model from ../config/bert-base/
2023-11-22 17:35:15 [INFO]: Using DataParallel model with 2 GPUs
2023-11-22 17:35:15 [INFO]: Start 2 train rounds of testexp at 2023-11-22 17:35:14.935758
2023-11-22 17:35:15 [INFO]: train_conf: {'pretrained_model': {'type': 'tape', 'location': '../config/bert-base/'}, 'data_parallel': True, 'backup': 'train.bak.{date}.tar.gz', 'rounds': [{'data': 'dash_vdjdb_mcpas', 'test_size': 0.2, 'batch_size': 256, 'n_epochs': 2, 'n_workers': 12, 'metrics': ['accuracy'], 'optimizer': {'type': 'adamw', 'lr': 0.0001}, 'train_bert_encoders': [-10, None], 'early_stopper': {'monitor': 'accuracy', 'patience': 1}, 'model_checkpoint': {'chk': 'train.{round}.model_{epoch}.chk', 'monitor':

Validating in epoch 0/2: 100%|██████████| 20/20 [00:11<00:00,  1.79batch/s]

2023-11-22 17:36:27 [INFO]: [EvalScoreRecoder]: In epoch 0/2, loss train score: 0.6785940407197687, val score: 0.6479369670152664
2023-11-22 17:36:27 [INFO]: [EvalScoreRecoder]: In epoch 0/2, accuracy train score: 0.5667649090747013, val score: 0.6095988948170732
2023-11-22 17:36:27 [INFO]: [EarlyStopper]: In epoch 0/2, accuracy score: 0.6095988948170732, best accuracy score: -inf;update best score to 0.6095988948170732
2023-11-22 17:36:27 [INFO]: [ModelCheckpoint]: Checkpoint at epoch 0: accuracy improved from -inf to 0.6095988948170732, saving model to ../output/testexp/train.0.model_0.chk



Validating in epoch 1/2: 100%|██████████| 20/20 [00:11<00:00,  1.79batch/s]

2023-11-22 17:37:37 [INFO]: [EvalScoreRecoder]: In epoch 1/2, loss train score: 0.588107004195829, val score: 0.5902026206254959
2023-11-22 17:37:37 [INFO]: [EvalScoreRecoder]: In epoch 1/2, accuracy train score: 0.6654024224460688, val score: 0.6545826981707317
2023-11-22 17:37:37 [INFO]: [EarlyStopper]: In epoch 1/2, accuracy score: 0.6545826981707317, best accuracy score: 0.6095988948170732;update best score to 0.6545826981707317
2023-11-22 17:37:37 [INFO]: [ModelCheckpoint]: Checkpoint at epoch 1: accuracy improved from 0.6095988948170732 to 0.6545826981707317, saving model to ../output/testexp/train.0.model_1.chk


...

Validating in epoch 0/2: 100%|██████████| 20/20 [00:11<00:00,  1.81batch/s]

2023-11-22 17:38:47 [INFO]: [EvalScoreRecoder]: In epoch 0/2, loss train score: 0.5022270385223099, val score: 0.45719517320394515
2023-11-22 17:38:47 [INFO]: [EvalScoreRecoder]: In epoch 0/2, accuracy train score: 0.7348854798092351, val score: 0.7677877286585366
2023-11-22 17:38:47 [INFO]: [EarlyStopper]: In epoch 0/2, accuracy score: 0.7677877286585366, best accuracy score: -inf;update best score to 0.7677877286585366
2023-11-22 17:38:47 [INFO]: [ModelCheckpoint]: Checkpoint at epoch 0: accuracy improved from -inf to 0.7677877286585366, saving model to ../output/testexp/train.1.model_0.chk



Validating in epoch 1/2: 100%|██████████| 20/20 [00:11<00:00,  1.81batch/s]

2023-11-22 17:39:54 [INFO]: [EvalScoreRecoder]: In epoch 1/2, loss train score: 0.4050456447691857, val score: 0.4712764069437981
2023-11-22 17:39:54 [INFO]: [EvalScoreRecoder]: In epoch 1/2, accuracy train score: 0.8037354864949189, val score: 0.774780868902439
2023-11-22 17:39:54 [INFO]: [EarlyStopper]: In epoch 1/2, accuracy score: 0.774780868902439, best accuracy score: 0.7677877286585366;update best score to 0.774780868902439
2023-11-22 17:39:54 [INFO]: [ModelCheckpoint]: Checkpoint at epoch 1: accuracy improved from 0.7677877286585366 to 0.774780868902439, saving model to ../output/testexp/train.1.model_1.chk


2023-11-22 17:39:54 [INFO]: [EvalScoreRecoder]: loss train socres: [0.5022270385223099, 0.4050456447691857], val scores: [0.45719517320394515, 0.4712764069437981]
2023-11-22 17:39:54 [INFO]: [EvalScoreRecoder]: accuracy train socres: [0.7348854798092351, 0.8037354864949189], val scores: [0.7677877286585366, 0.774780868902439]
2023-11-22 17:39:54 [INFO]: End training...
2023-11-22 17:39:54 [INFO]: ======================
2023-11-22 17:39:54 [INFO]: 1 train round result: {'metrics': ['accuracy'], 'train.score': OrderedDict([('loss', [0.5022270385223099, 0.4050456447691857]), ('accuracy', [0.7348854798092351, 0.8037354864949189])]), 'val.score': OrderedDict([('loss', [0.45719517320394515, 0.4712764069437981]), ('accuracy', [0.7677877286585366, 0.774780868902439])]), 'n_epochs': 2, 'stopped_epoch': 1, 'monitor': 'accuracy', 'best_epoch': 1, 'best_score': 0.774780868902439, 'best_chk': '../output/testexp/train.1.model_1.chk'}, writing to ../output/testexp/train.1.result.json
2023-11-22 17:39